<a href="https://colab.research.google.com/github/Jeisha/FYP-Temp/blob/main/PART_1_Car_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('drive/MyDrive/FYP')

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np
import glob
import cv2
import glob
%matplotlib inline

In [ ]:
car_detect = load_model(os.path.join('Model','car_detector.h5'))
paths = {
    'IMAGE_WORK_PATH': os.path.join('image_work'),
    'CAR_DETECTED_PATH':os.path.join('image_work','CAR_DETECTED')
 }

In [ ]:
def rescale_frame_car_detection(frame):
    width = 500
    height = 500
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)[200:350,170:370]

In [ ]:
cap = []
for i in glob.glob(os.path.join(os.getcwd(),'Video','*.mp4')):
  cap.append(cv2.VideoCapture(i))

In [ ]:
car_no = 0
count = 0
for vid in range(len(cap)):
    while True:
        ret,frame = cap[vid].read()
        if ret:
            if count %10 == 0:
                cars = []
                car_detection = rescale_frame_car_detection(frame)
                car = cv2.cvtColor(car_detection, cv2.COLOR_BGR2RGB) #convert to RGB Channel
                car = cv2.resize(car, (224, 224)) # resize image
                car = img_to_array(car) #convert image to array
                car = preprocess_input(car) # preprocess the array
                cars.append(car)
                if len(cars) > 0:
                    cars = np.array(cars, dtype="float32") #convert array to float 
                    pred = car_detect.predict(cars,batch_size=32)
                car, no_car = pred[0][0],pred[0][1]
                if car > no_car:
                    img_item = os.path.join(paths['CAR_DETECTED_PATH'],f'car-{car_no:05d}.png')
                    cv2.imwrite(img_item, frame)
                    car_no +=1
            count += 1
        else:
            cap[vid].release()
            break